# Introduction
In this kernal, I will use TF-IDF to vectorize the [articles data](https://www.kaggle.com/snapcrack/all-the-news/data) and cluster them. 
Then, I will make paper recommendation.

**1. Input data**

In [1]:
import numpy as np
import pandas as pd 
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

text = pd.read_csv('../input/articles1.csv')
text.head()

FileNotFoundError: ignored

**2. Using TF-IDF vectorize the articles**

[Introduction](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) to TF-IDF

In [0]:
text_content = text['content']
vector = TfidfVectorizer(stop_words = 'english')
tfidf = vector.fit_transform(text_content)
text_content.head()

**3. Using elbow method to decide cluster number**

Introduction to [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)

In [0]:
K = range(1,15)
SSE = []
for k in K:
    kmeans = MiniBatchKMeans(n_clusters = k,batch_size = 300)
    kmeans.fit(tfidf)
    SSE.append(kmeans.inertia_)
    
import matplotlib.pyplot as plt
plt.plot(K,SSE,'bx-')
plt.title('Elbow Method')
plt.xlabel('cluster numbers')
plt.show()



**5. Using MiniBatchKMean to cluster**

[Comparison of the K-Means and MiniBatchKMeans clustering algorithms](http://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html#sphx-glr-auto-examples-cluster-plot-mini-batch-kmeans-py)

In [0]:
k = 4
kmeans = MiniBatchKMeans(n_clusters = k)
kmeans.fit(tfidf)
centers = kmeans.cluster_centers_.argsort()[:,::-1]
terms = vector.get_feature_names()

for i in range(0,k):
    word_list=[]
    print("cluster%d:"% i)
    for j in centers[i,:15]:
        word_list.append(terms[j])
    print(word_list) 


**6. Article Recommendation**

Since we had vectorize the articles by TF-IDF, we only need to compare articles by comparing their vectors. Bigger inner product of two vectors means they shares more similar information. Here we take the first article in artciles3.csv--Alton Sterling’s son: ’Everyone needs to protest the right way, with peace’ as example.

In [0]:
similarity = np.dot(tfidf[0],np.transpose(tfidf))
x = np.array(similarity.toarray()[0])
print(text['title'][0])
print('\nsimiliar papers:')
print('\n'.join(text['title'].loc[np.argsort(x)[-7:-2]]))


**7. Topic modeling **

Here I use NMF instead of LDA because LDA's calculation is time-consuming. 

The major topics consist of politics, society, finance and internetional affairs, which is quite similiar to our cluster resu

In [0]:
from sklearn.decomposition import NMF

nmf  = NMF(n_components = 4)
nmf.fit(tfidf)
for i in range(0,k):
    word_list=[]
    print("Topic%d:"% i)
    for j in nmf.components_.argsort()[i,-16:-1]:
        word_list.append(terms[j])
    print(word_list)